# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'ARTN/(G OAEQEE LAAD BSGLNSCTLKMS)',
 'calculationPrice': 'close',
 'open': None,
 'openTime': None,
 'openSource': 'ffciliao',
 'close': None,
 'closeTime': None,
 'closeSource': 'laciioff',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 135.13,
 'latestSource': 'Close',
 'latestTime': 'January 14, 2021',
 'latestUpdate': 1622721209519,
 'latestVolume': None,
 'iexRealtimePrice': 0,
 'iexRealtimeSize': 0,
 'iexLastUpdated': 0,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 135.79,
 'previousVolume': 91323617,
 'change': -2.01,
 'changePercent': -0.01572,
 'volume': None,
 'iexMarketPercent': None,
 'iexVolume': 0,
 'avgTotalVolume': 115017759,
 'iexBidPri

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

39.4

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [10]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,128.680,56.8,N/A
1,AAL,16.490,-1.19,N/A
2,AAP,170.320,25.31,N/A
3,AAPL,131.140,41,N/A
4,ABBV,111.820,24.94,N/A
...,...,...,...,...
500,YUM,111.420,31.73,N/A
501,ZBH,167.100,-221.85,N/A
502,ZBRA,428.430,47.13,N/A
503,ZION,53.427,22.24,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [12]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,40.57,2.48,N/A
1,NLOK,20.64,4.1,N/A
2,AIV,5.11,4.82,N/A
3,BIO,617.58,5.3,N/A
4,UNM,26.03,5.66,N/A
5,ALL,113.97,7.66,N/A
6,AFL,47.17,7.7,N/A
7,EBAY,56.23,7.97,N/A
8,CPB,47.74,7.98,N/A
9,KIM,16.79,8.47,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [13]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [14]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [15]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,40.57,2.48,4929
1,NLOK,20.64,4.1,9689
2,AIV,5.11,4.82,39138
3,BIO,617.58,5.3,323
4,UNM,26.03,5.66,7683
5,ALL,113.97,7.66,1754
6,AFL,47.17,7.7,4239
7,EBAY,56.23,7.97,3556
8,CPB,47.74,7.98,4189
9,KIM,16.79,8.47,11911


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [16]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [21]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,129.120,N/A,57,N/A,8.19,N/A,7.28,N/A,35.056985,N/A,14.075722,N/A,N/A
1,AAL,17.050,N/A,-1.21,N/A,-1.9,N/A,0.4112,N/A,-6.009100,N/A,1.346053,N/A,N/A
2,AAP,173.780,N/A,24.57,N/A,3.07,N/A,1.18,N/A,11.722599,N/A,2.726824,N/A,N/A
3,AAPL,133.750,N/A,40.6,N/A,33.24,N/A,8.1,N/A,28.072392,N/A,20.843988,N/A,N/A
4,ABBV,117.000,N/A,25,N/A,13.4,N/A,4.96,N/A,17.237073,N/A,9.935436,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.170,N/A,31.46,N/A,-4.15,N/A,5.9,N/A,24.103301,N/A,10.043133,N/A,N/A
501,ZBH,166.500,N/A,-231.39,N/A,2.95,N/A,4.82,N/A,27.933770,N/A,7.985689,N/A,N/A
502,ZBRA,422.740,N/A,48.88,N/A,11.77,N/A,5.23,N/A,32.667139,N/A,12.344032,N/A,N/A
503,ZION,53.356,N/A,22.07,N/A,1.22,N/A,2.77,N/A,12.767868,N/A,3.346259,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [18]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,209.57,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,245.95,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12.00,N/A,10.18,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,32.59,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.27,N/A,14.49,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,53.86,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,30.73,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,160.94,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1753.82,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.44,N/A,32.3,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [22]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [23]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [24]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0       0.869307
1       0.154455
2        0.49505
3       0.780198
4        0.50297
         ...    
500     0.642574
501    0.0257426
502     0.827723
503     0.449505
504     0.817822
Name: PE Percentile, Length: 505, dtype: object
0       0.742574
1      0.0475248
2       0.411881
3       0.960396
4       0.865347
         ...    
500    0.0455446
501      0.39802
502     0.837624
503     0.122772
504     0.924752
Name: PB Percentile, Length: 505, dtype: object
0        0.80495
1      0.0257426
2       0.159406
3       0.843564
4       0.692079
         ...    
500     0.742574
501     0.684158
502     0.708911
503     0.432673
504     0.930693
Name: PS Percentile, Length: 505, dtype: object
0      0.853465
1      0.029703
2      0.273267
3       0.79802
4      0.512871
         ...   
500    0.722772
501     0.79604
502    0.839604
503    0.340594
504    0.841584
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.819802
1      0.0514851
2       0.122772
3       0.940

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,129.120,N/A,57.00,0.869307,8.19,0.742574,7.2800,0.80495,35.056985,0.853465,14.075722,0.819802,N/A
1,AAL,17.050,N/A,-1.21,0.154455,-1.90,0.0475248,0.4112,0.0257426,-6.009100,0.029703,1.346053,0.0514851,N/A
2,AAP,173.780,N/A,24.57,0.49505,3.07,0.411881,1.1800,0.159406,11.722599,0.273267,2.726824,0.122772,N/A
3,AAPL,133.750,N/A,40.60,0.780198,33.24,0.960396,8.1000,0.843564,28.072392,0.79802,20.843988,0.940594,N/A
4,ABBV,117.000,N/A,25.00,0.50297,13.40,0.865347,4.9600,0.692079,17.237073,0.512871,9.935436,0.655446,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.170,N/A,31.46,0.642574,-4.15,0.0455446,5.9000,0.742574,24.103301,0.722772,10.043133,0.663366,N/A
501,ZBH,166.500,N/A,-231.39,0.0257426,2.95,0.39802,4.8200,0.684158,27.933770,0.79604,7.985689,0.50495,N/A
502,ZBRA,422.740,N/A,48.88,0.827723,11.77,0.837624,5.2300,0.708911,32.667139,0.839604,12.344032,0.760396,N/A
503,ZION,53.356,N/A,22.07,0.449505,1.22,0.122772,2.7700,0.432673,12.767868,0.340594,3.346259,0.176238,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.320,N/A,57.20,0.867327,8.00,0.731683,7.5000,0.811881,34.937202,0.853465,13.811216,0.80396,0.813663
1,AAL,16.464,N/A,-1.21,0.154455,-1.80,0.0475248,0.3982,0.0257426,-6.217273,0.029703,1.382275,0.0534653,0.0621782
2,AAP,172.810,N/A,25.71,0.510891,3.20,0.425743,1.2300,0.174257,11.697642,0.267327,2.596165,0.116832,0.29901
3,AAPL,134.840,N/A,41.22,0.786139,34.20,0.962376,8.0600,0.843564,29.598378,0.815842,21.869015,0.950495,0.871683
4,ABBV,115.820,N/A,24.74,0.49901,13.29,0.855446,4.9500,0.688119,16.818743,0.49901,9.859262,0.649505,0.638218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.230,N/A,31.66,0.648515,-4.28,0.0435644,5.9700,0.746535,24.052222,0.716832,10.517685,0.685149,0.568119
501,ZBH,163.210,N/A,-230.26,0.0257426,2.93,0.394059,5.0100,0.69703,27.664835,0.794059,8.327366,0.536634,0.489505
502,ZBRA,415.960,N/A,48.69,0.819802,11.89,0.843564,5.2100,0.706931,31.952744,0.833663,11.873153,0.742574,0.789307
503,ZION,52.700,N/A,22.06,0.451485,1.18,0.120792,2.7000,0.420792,12.546571,0.326733,3.163156,0.160396,0.29604


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [25]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [26]:
portfolio_input()

Enter the value of your portfolio:10000000


In [28]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\PLT\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,129.12,1548,57.000000,0.869307,8.190000,0.742574,7.280000,0.80495,35.056985,0.853465,14.075722,0.819802,N/A
1,NTRS,100.79,1984,17.300000,0.346535,1.960000,0.272277,3.260000,0.510891,11.865420,0.283168,3.262144,0.164356,N/A
2,NTAP,67.04,2983,21.540000,0.439604,40.960000,0.972277,2.730000,0.426733,11.883103,0.287129,3.582808,0.19604,N/A
3,NSC,260.69,767,32.910000,0.673267,4.410000,0.534653,6.690000,0.782178,16.996838,0.50099,7.802817,0.493069,N/A
4,NRG,41.12,4863,2.480000,0.188119,5.280000,0.614851,1.110000,0.146535,7.936363,0.10495,5.460136,0.324752,N/A
5,NOW,520.02,384,156.420000,0.986139,38.210000,0.968317,23.920000,0.99604,197.434179,0.99802,29.512472,0.990099,N/A
6,NOV,16.06,12453,-2.370000,0.138614,1.150000,0.105941,0.875500,0.0851485,25969.403118,1,7.564639,0.473267,N/A
7,NOC,311.80,641,21.790000,0.443564,5.000000,0.583168,1.440000,0.211881,11.585538,0.263366,8.293294,0.534653,N/A
8,NLSN,23.01,8691,-53.390000,0.0613861,4.020000,0.515842,1.330000,0.19505,9.601851,0.156436,4.367231,0.239604,N/A
9,NLOK,21.24,9416,4.000000,0.190099,-26.280000,0.0158416,6.560000,0.768317,17.099639,0.50297,8.316727,0.536634,N/A


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [29]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [30]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [31]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [32]:
writer.save()